In [43]:

from variables import sourcepath, yaml_path
import os
import pandas as pd
import numpy as np
import etl.dim_author as auth
import etl.dim_journal as jour
import etl.common_functions as cof
import etl.dim_paper as pape
import etl.database as db
from db_credentials import dwh_db_connection_params
import roman
import datetime
import yaml

In [57]:
eng, psycop2connect=db.initialize_engine(connection_params=dwh_db_connection_params)

In [ ]:
for file in os.listdir(yaml_path):
    if file.endswith('.yaml'):
        with open(yaml_path + '/' + file, encoding='UTF-8') as f:
            yaml_data = yaml.load_all(f, Loader=yaml.SafeLoader)
            df=
        #for figure_data in json_data:

In [6]:
df_list=[]
with open(os.path.join(yaml_path, 'conceptual_methods.yaml'), 'r') as f:
    for doc in yaml.load_all(f, Loader=yaml.SafeLoader):
        df=pd.json_normalize(doc)
        df_list.append(df)

In [9]:
df_list[1]

,conceptual method
0,"[{':type': 'conceptual'}, {':altLabel': ['conc..."


In [2]:
entities=pd.read_csv(os.path.join(sourcepath, 'entities.csv'))

In [3]:
entities.columns

Index(['article_id', 'para_id', 'sentence_start', 'sentence_id',
       'journal_akronym', 'last_section_title', 'last_subsection_title',
       'sentence_type', 'sentence', 'entity', 'label', 'ent_id', 'ent_path',
       'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6',
       'level_7', 'level_8'],
      dtype='object')

In [4]:
for_map_and_dim=entities[['label', 'ent_id', 'ent_path']]
for_map_and_dim.head(10)

,label,ent_id,ent_path
0,TOPIC,knowledge repository,IS topic/knowledge management/knowledge reposi...
1,TOPIC,digital platform,IS topic/digital business/digital platform
2,TECHNOLOGY,Salesforce.com,IS technology/enterprise information system/op...
3,CONCEPTUAL_METHOD,participatory design,conceptual method/design science/design method...
4,PARTICIPANTS,group participant,participant/group participant
5,CONCEPTUAL_METHOD,participatory design,conceptual method/design science/design method...
6,TECHNOLOGY,blog,IS technology/social media/blog
7,TECHNOLOGY,blog,IS technology/social media/blog
8,TECHNOLOGY,online community,IS technology/social media/online community
9,TECHNOLOGY,Reddit,IS technology/social media/social media site/R...


In [5]:
for_map_and_dim.shape

(2137618, 3)

In [32]:
for_map_and_dim.drop_duplicates(inplace=True)
for_map_and_dim.shape

/home/luisa_waack/virtualenvs/thesis3.9/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(1966, 3)

In [15]:
hierarchies=[]
for path in for_map_and_dim.ent_path:
    path_as_list=path.split('/')
    for idx1, parent_ent in enumerate(path_as_list):
        for idx2, child_ent in enumerate(path_as_list):
            if idx2-idx1 >-1:
                if idx1==0:
                    highest_parent_flag=True
                else:
                    highest_parent_flag=False
                parent_child_relations=[parent_ent, child_ent, (idx2-idx1), highest_parent_flag]
                hierarchies.append(parent_child_relations)


In [30]:
map_entity_hierarchy=pd.DataFrame(hierarchies, columns=['parent', 'child', 'depth_from_parent', 'highest_parent_flag']).drop_duplicates()
#get list of leaf nodes (those that have no child entities but themselves)
lowest_children=map_entity_hierarchy.parent.value_counts().loc[lambda x: x==1].index.to_list()
#assign lowest child flag based on this list
map_entity_hierarchy['lowest_child_flag']=map_entity_hierarchy['child'].apply(lambda c: True if c in lowest_children else False)

In [36]:
map_entity_hierarchy

,parent,child,depth_from_parent,highest_parent_flag,lowest_child_flag
0,IS topic,IS topic,0,True,False
1,IS topic,knowledge management,1,True,False
2,IS topic,knowledge repository,2,True,True
3,knowledge management,knowledge management,0,False,False
4,knowledge management,knowledge repository,1,False,True
...,...,...,...,...,...
20973,sequence mining,sequence mining,0,False,True
20977,economic sector,food law,3,True,True
20980,manufacturing industry,food law,2,False,True
20982,food industry,food law,1,False,True


In [34]:
#now generate primary keys
dim_entity=for_map_and_dim[['label', 'ent_id']].drop_duplicates()

In [37]:
dim_entity['entity_pk']=list(range(1, 1+dim_entity.index.size))
dim_entity

,label,ent_id,entity_pk
0,TOPIC,knowledge repository,1
1,TOPIC,digital platform,2
2,TECHNOLOGY,Salesforce.com,3
3,CONCEPTUAL_METHOD,participatory design,4
4,PARTICIPANTS,group participant,5
...,...,...,...
2046449,SECTOR,event industry,1961
2051619,ANALYSIS_METHOD,autoregressive conditional heteroscedasticity ...,1962
2106336,ANALYSIS_METHOD,autoencoder,1963
2125258,ANALYSIS_METHOD,sequence mining,1964


In [38]:
dim_entity.ent_id[2051619]

'autoregressive conditional heteroscedasticity model'

In [44]:
#get parent pk
map_entity_hierarchy=pd.merge(map_entity_hierarchy, dim_entity, how='left', left_on='parent', right_on='ent_id').drop(columns=['label', 'ent_id']).rename(columns={'entity_pk': 'parent_entity_pk'})

In [45]:
print(map_entity_hierarchy.shape)
map_entity_hierarchy.dropna(axis=0, how='any', inplace=True)
print(map_entity_hierarchy.shape)

(8058, 6)
(7354, 6)
